In [1]:
from z3 import *
import time

# Parameters
H = 4  # Height of lattice (e.g., 4 rows)
L = 5  # Width of lattice (e.g., 3 columns)
d = 5  # Target code distance

# Initialize Z3 solver
solver = Solver()
SyndromeRows = []

# Create 8 Boolean variables per stabilizer (excluding center of 3x3)
# Apply to each row from 2 to H-1 (i.e., rows where stabilizers are used)
for i in range(H - 2):
    current = [Bool(f"s_{i}_{j}") for j in range(8)]
    SyndromeRows.append(current)
    # Constraint: at most 3 active cells per stabilizer
    solver.add(Sum([If(bit, 1, 0) for bit in current]) <= 3)

# Precompute relative positions of the 8 neighbors (excluding center)
# Order: top-left to bottom-right, skipping center (1,1)
neighbor_offsets = [
    (-2, -1), (-2, 0), (-2, +1),
    (-1, -1), (-1, 0), (-1, +1),
    (0, -1),           (0, +1)
]

# Loop over all nonzero 2-row inputs (2*L bits)
for i in range(1, 2 ** (2 * L)):
    codeword = []

    # Initial 2 rows
    bits = [bool(int(b)) for b in format(i, f"0{2 * L}b")]
    for b in bits:
        codeword.append(BoolVal(b))

    # Compute the rest using stabilizers
    for n in range(2, H):
        for m in range(L):
            terms = []
            stab = SyndromeRows[n - 2]
            for idx, (dn, dm) in enumerate(neighbor_offsets):
                nn = n + dn
                mm = (m + dm + L) % L  # periodic boundary in horizontal
                if 0 <= nn < H:
                    idx_flat = nn * L + mm
                    if idx_flat < len(codeword):
                        terms.append(And(codeword[idx_flat], stab[idx]))
            # XOR all terms
            value = False
            for t in terms:
                value = t if value is False else Xor(value, t)
            codeword.append(value)

    # Enforce Hamming weight ≥ d
    solver.add(Sum([If(b, 1, 0) for b in codeword]) >= d)

start = time.time()
if solver.check() == sat:
    model = solver.model()
    print("SAT")
    for i, row in enumerate(SyndromeRows):
        print(f"Stabilizer shape for row {i+2}:")
        shape = ['●' if is_true(model[b]) else '·' for b in row]
        print(f"{shape[0]} {shape[1]} {shape[2]}")
        print(f"{shape[3]} {shape[4]} {shape[5]}")
        print(f"{shape[6]}   {shape[7]}")
        print()
else:
    print("UNSAT")

print("Elapsed time:", round(time.time() - start, 2), "seconds")


SAT
Stabilizer shape for row 2:
● · ●
● · ·
·   ·

Stabilizer shape for row 3:
● · ●
· · ●
·   ·

Elapsed time: 0.58 seconds
